<a href="https://colab.research.google.com/github/u-keigo/RRIproject1/blob/main/LSTM_hurst3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ハースト指数を予測する
## （バッチ化に挑戦）
### 失敗した

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from glob import glob
import pandas as pd
import linecache


def read_Gauss(name):
  nums = []  # 整数を入れるリスト
  with open(name, 'r', encoding='utf-8') as fin:  # ファイルを開く
    for line in fin.readlines():  # 行を読み込んでfor文で回す
        try:
            line = line.replace('\n','')
            num = float(line)  # 行を整数（int）に変換する
        except ValueError as e:
            print(e, file=sys.stderr)  # エラーが出たら画面に出力
            continue

        nums.append(num)  # 変換した整数をリストに保存する
  return (nums)


# カテゴリを配列で取得
drive_dir = "/content/drive/My Drive/python/"

categories = [name for name in os.listdir(drive_dir + 'data_gauss') if os.path.isdir(drive_dir + "data_gauss/" +name)]
print(categories)



['H=0.1', 'H=0.9', 'H=0.7', 'H=0.8', 'H=0.6', 'H=0.5', 'H=0.4', 'H=0.3', 'H=0.2']


In [3]:
import numpy as np
DAT = pd.DataFrame(np.zeros(shape=(1024, (32*9))))
i=0
for cat in categories:
    path = drive_dir + "data_gauss/" + cat + "/*.rri"
    files = glob(path)
    for text_name in files:
      data = pd.Series(read_Gauss(text_name))
      DAT.iloc[:, i] = data
      DAT.rename(columns={i: cat}, inplace=True)
      i = i+1


# データフレームシャッフル
DAT = DAT.sample(frac=1,axis=1).reset_index(drop=True)
DAT.T.head()

KeyboardInterrupt: ignored

In [ ]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch.nn.functional as F
from torch import optim


category2index = {}
for cat in categories:
    if cat in category2index: continue
    category2index[cat] = len(category2index)
print(category2index)


def category2tensor(cat):
    return torch.tensor([category2index[cat]], dtype=torch.long)

# データフレームの形状変更・indexの要素化
df = DAT.T
df['index'] = df.index

df_label = df.loc[:, 'index']
df_data = df.drop('index', axis=1)

dlen = len(DAT.columns)  # 288
train_x, test_x, train_y, test_y = train_test_split(df_data, df_label, test_size=0.3, random_state=0)


train_x2 = torch.FloatTensor(train_x.values)
test_x2 = torch.FloatTensor(test_x.values)
train_y2, _ = pd.factorize(train_y)   # カテゴリー変数を数値に変換
test_y2, _ = pd.factorize(test_y)

train_y2 = torch.FloatTensor(train_y2)
test_y2 = torch.FloatTensor(test_y2)

# 次元を増やさないといけない？
train_x2 = train_x2.unsqueeze(2)
train_y2 = train_y2.unsqueeze(1)
test_x2 = test_x2.unsqueeze(2)
test_y2 = test_y2.unsqueeze(1)


# 特徴量とラベルを結合したデータセットを作成
train_dataset = TensorDataset(train_x2, train_y2)
test_dataset = TensorDataset(test_x2, test_y2)


# DataLoaderを使って、データセットを128個のミニパッチに分ける
# ミニパッチサイズを指定したデータローダを作成
train_batch = DataLoader(
    dataset = train_dataset,   # データセットの指定
    batch_size = 32,   # バッチサイズの指定
    shuffle = True,    # シャッフルするかどうかの指定
    num_workers = 2)   # コアの数

test_batch = DataLoader(
    dataset = test_dataset,   # データセットの指定
    batch_size = 32,   # バッチサイズの指定
    shuffle = False,    # シャッフルするかどうかの指定
    num_workers = 2)   # コアの数


# ミニバッチデータセットの確認
for data, label in train_batch:
  print("batch data size: {}". format(data.size()))  # バッチの入力データサイズ
  print("batch label size: {}". format(label.size()))   # バッチのラベルサイズ
  break


In [ ]:
# nn.Moduleを継承して新しいクラスを作る。決まり文句
class LSTMClassifier(nn.Module):
    # モデルで使う各ネットワークをコンストラクタで定義
    def __init__(self, embedding_dim, hidden_dim, tagset_size):
        # 親クラスのコンストラクタ。決まり文句
        super(LSTMClassifier, self).__init__()
        # 隠れ層の次元数。これは好きな値に設定しても行列計算の過程で出力には出てこないので。
        self.hidden_dim = hidden_dim
        # LSTMの隠れ層。これ１つでOK。超便利。
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)   # inputsの最初の次元がバッチサイズとして認識される
        # LSTMの出力を受け取って全結合してsoftmaxに食わせるための１層のネットワーク
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

        self.softmax = nn.LogSoftmax()

    # 順伝播処理はforward関数に記載
    def forward(self, embeds):
        # 2次元テンソルをLSTMに食わせられる様にviewで３次元テンソルにした上でLSTMへ流す。
        # 上記で説明した様にmany to oneのタスクを解きたいので、第二戻り値だけ使う。
        # embeds.size() = (batch_size × len(sentence) × embedding_dim)
        _, lstm_out = self.lstm(embeds)
        # lstm_out[0]は３次元テンソルになってしまっているので2次元に調整して全結合。
        tag_space = self.hidden2tag(lstm_out[0])
        # tag_space.size() = (1 × batch_size × tagset_size)

        # softmaxに食わせて、確率として表現
        # (batch_size × tagset_size)にするためにsqueeze()する
        tag_scores = self.softmax(tag_space.squeeze())
        # tag_scores.size() = (batch_size × tagset_size)

        return tag_scores

category2index = {}
for cat in categories:
    if cat in category2index: continue
    category2index[cat] = len(category2index)
print(category2index)

def category2tensor(cat):
    return torch.tensor([category2index[cat]], dtype=torch.long)


In [ ]:
# 入力次元数
EMBEDDING_DIM = 1
# 隠れ層の次元数
HIDDEN_DIM = 128
# 分類先のカテゴリの数
TAG_SIZE = len(categories)

# ネットワークのロード
# CPUとGPUのどちらを使うかを指定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, TAG_SIZE).to(device)
# デバイスの確認
print("Device: {}".format(device))


loss_function = nn.MSELoss()  # 損失関数（平均二乗誤差：MSE）
# 最適化関数の定義
# optimizer = optim.Adam(net.parameters())
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 損失を保存するリストを作成
train_loss_list = []  # 学習損失
test_loss_list = []  # 評価損失

epoch = 100

for i in range(epoch):
  all_loss = 0
  # エポックの進行状況を表示
  print('-------------------------------------')
  print("Epoch: {}/{}".format(i+1,epoch))

  # 損失の初期化
  train_loss = 0  # 学習損失
  test_loss = 0  # 評価損失

  # ---学習パート--- #
  # ニューラルネットワークを学習モードに設定
  net.train()
  for j, (xx, yy) in enumerate(train_batch):
        # モデルが持ってる勾配の情報をリセット
        net.zero_grad()
        inputs = xx.cuda()
        # GPUにTensorを転送
        inputs = inputs.to(device)
        # 順伝播の結果を受け取る
        out = net(inputs)
        answer = yy.cuda()
        # GPUにTensorを転送
        answer = answer.to(device)
        # 正解とのlossを計算
        loss = loss_function(out, answer)
        # 勾配をセット
        loss.backward()
        # 逆伝播でパラメータ更新
        optimizer.step()
        # lossを集計
        train_loss += loss.item()
  
  # ミニバッチの平均の損失を計算
  batch_train_loss = train_loss / len(train_batch)
  # ---学習パートはここまで--- #

  # ---評価パート--- #
  # ニューラルネットワークを評価モードに設定
  net.eval()
  # 評価時の計算で自動微分機能をオフにする
  with torch.no_grad():
    for j, (xx, yy) in enumerate(test_batch):
        inputs = xx.cuda()
        # GPUにTensorを転送
        inputs = inputs.to(device)
        # 順伝播の結果を受け取る
        out = net(inputs)
        answer = yy.cuda()
        # GPUにTensorを転送
        answer = answer.to(device)
        # 正解とのlossを計算
        loss = loss_function(out, answer)
        # lossを集計
        test_loss += loss.item()

    
  # ミニバッチの平均の損失を計算
  batch_test_loss = test_loss / len(test_batch)
  # ---評価パートはここまで--- #

  # エポックごとに損失を表示
  print("Train_Loss : {:.2E} Test_Loss: {:.2E}".format(
      batch_train_loss, batch_test_loss))
  # 損失をリスト化して保存
  train_loss_list.append(batch_train_loss)
  test_loss_list.append(batch_test_loss)
print("done.")



In [ ]:
import matplotlib.pyplot as plt
# 損失
# 損失
fig2 = plt.figure()
plt.title('Train and Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(range(1, epoch+1), train_loss_list, color='blue',
         linestyle='-', label='Train_Loss')
plt.plot(range(1, epoch+1), test_loss_list, color='red',
         linestyle='--', label='Test_Loss')
plt.legend()
plt.show()
